<a href="https://colab.research.google.com/github/Shakilkhan24/Playground_DL/blob/main/langchain_simple_introductory_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('I just followed a sample sequential way of langchain chatbot building')
print('i did not run on any data or sources , even i did not run the code, Just arranged all together' )
print('as if i can work on sequentially later and do powerfull stuffs on each point')

I just followed a sample sequential way of langchain chatbot building
i did not run on any data or sources , even i did not run the code, Just arranged all together
as if i can work on sequentially later and do powerfull stuffs on each point


## sample pipeline for langchain
llm > promt > external sources > retriever > vector_stores > tools (retrieving, web_searching) > agent(tools) > fast_api > prompts_settings for agents_ to run

In [ ]:
!pip install langchain langchain-openai chromadb tavily

from typing import List

# for deployment possibly
from fastapi import FastAPI
# ____________________________________________________________________ #
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
# tool
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
# _____________________________________________________________________ #
from langchain_openai import ChatOpenAI
from langchain import hub
# agent
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
# _____________________________________________________________________ #
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes

# 1. Load Retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]


# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)




```
# Deal with prompt templates for retrieving datas from
external sources like documents or webpage.
```



In [ ]:
# 5. AgentExecuter(some input output prompts)

class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )


class Output(BaseModel):
    output: str

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)